<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/382_GCO_Prioritization_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Prioritization utilities for Governance & Compliance Orchestrator

Prioritizes compliance issues by severity, urgency, and impact.
"""

from typing import Dict, Any, List
from config import GovernanceComplianceOrchestratorConfig


def calculate_priority_score(
    compliance_event: Dict[str, Any],
    events_lookup: Dict[str, Dict[str, Any]],
    config: GovernanceComplianceOrchestratorConfig
) -> float:
    """
    Calculate priority score for a compliance event.

    Args:
        compliance_event: Compliance event dict
        events_lookup: Lookup dict for events by event_id
        config: Configuration with priority scoring weights

    Returns:
        Priority score (0.0 to 100.0)
    """
    weights = config.priority_scoring_weights

    # Severity score (0-100)
    severity_map = {"critical": 100, "high": 75, "medium": 50, "low": 25}
    severity = compliance_event.get("severity", "medium")
    severity_score = severity_map.get(severity, 50)

    # Urgency score (based on risk type and severity)
    risk_type = compliance_event.get("risk_type", "policy_violation")
    if risk_type == "policy_violation" and severity == "critical":
        urgency_score = 100
    elif risk_type == "policy_violation" and severity == "high":
        urgency_score = 80
    elif severity == "critical":
        urgency_score = 90
    else:
        urgency_score = 50

    # Impact score (based on severity and required action)
    required_action = compliance_event.get("recommended_action", "")
    if "block" in required_action.lower() or "escalate" in required_action.lower():
        impact_score = 90
    elif "review" in required_action.lower():
        impact_score = 60
    else:
        impact_score = 40

    # Frequency score (simplified - could be enhanced with historical data)
    frequency_score = 50  # Default, could be calculated from event frequency

    # Calculate weighted priority score
    priority_score = (
        severity_score * weights.get("severity", 0.40) +
        urgency_score * weights.get("urgency", 0.30) +
        impact_score * weights.get("impact", 0.20) +
        frequency_score * weights.get("frequency", 0.10)
    )

    return round(priority_score, 2)


def prioritize_compliance_events(
    compliance_events: List[Dict[str, Any]],
    events_lookup: Dict[str, Dict[str, Any]],
    config: GovernanceComplianceOrchestratorConfig
) -> List[Dict[str, Any]]:
    """
    Prioritize compliance events by score.

    Args:
        compliance_events: List of compliance events
        events_lookup: Lookup dict for events by event_id
        config: Configuration with priority scoring weights

    Returns:
        List of prioritized compliance events (sorted by priority score, descending)
    """
    prioritized = []

    for event in compliance_events:
        priority_score = calculate_priority_score(event, events_lookup, config)

        # Get agent name from event if available
        event_id = event.get("event_id")
        agent_name = None
        if event_id and event_id in events_lookup:
            agent_name = events_lookup[event_id].get("agent_name")

        prioritized_event = event.copy()
        prioritized_event["priority_score"] = priority_score
        prioritized_event["agent_name"] = agent_name
        prioritized.append(prioritized_event)

    # Sort by priority score (descending)
    return sorted(prioritized, key=lambda x: x.get("priority_score", 0.0), reverse=True)


def generate_summary(
    agent_action_logs: List[Dict[str, Any]],
    compliance_events: List[Dict[str, Any]],
    bias_signals: List[Dict[str, Any]],
    drift_signals: List[Dict[str, Any]],
    risk_scores: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Generate summary statistics for the governance analysis.

    Args:
        agent_action_logs: List of agent action log events
        compliance_events: List of compliance events (violations)
        bias_signals: List of bias signals
        drift_signals: List of drift signals
        risk_scores: Risk scores dict

    Returns:
        Summary dict with key statistics
    """
    # Count violations by severity
    severity_counts = {"critical": 0, "high": 0, "medium": 0, "low": 0}
    for event in compliance_events:
        severity = event.get("severity", "medium")
        if severity in severity_counts:
            severity_counts[severity] += 1

    # Get unique agents affected
    agents_affected = set()
    for event in agent_action_logs:
        agent_name = event.get("agent_name")
        if agent_name:
            agents_affected.add(agent_name)

    # Get agents with violations
    agents_with_violations = set()
    for event in compliance_events:
        event_id = event.get("event_id")
        # Could enhance this by looking up the event to get agent_name
        # For now, we'll use a simpler approach
        pass

    return {
        "total_events_analyzed": len(agent_action_logs),
        "total_violations": len(compliance_events),
        "high_severity_count": severity_counts.get("high", 0) + severity_counts.get("critical", 0),
        "critical_severity_count": severity_counts.get("critical", 0),
        "bias_signals_count": len(bias_signals),
        "drift_signals_count": len(drift_signals),
        "agents_affected": sorted(list(agents_affected)),
        "overall_risk_score": risk_scores.get("overall_risk_score", 0.0),
        "severity_breakdown": severity_counts
    }

